In [2]:
import json

# Step 1: Specify the file path
file_path = "./train_data.json"  # Replace with your actual file path

# Step 2: Open the file in read mode
try:
  with open(file_path, "r") as json_file:
    # Step 3: Load the JSON data using json.load()
    training_set = json.load(json_file)
except FileNotFoundError:
  print(f"Error: File not found at {file_path}")
else:
  print("JSON data parsed successfully!")
  # Step 4: Access and process the data
  # (See examples below based on data structure)

print(training_set[0])


JSON data parsed successfully!
{'data': {'cmbd': 'null', 'id': '19026587', 'docid': 'null', 'page': 'null', 'paragraph': 'null', 'text': " nº historia clinica: ** *** *** nºepisodi: ******** sexe: home data de naixement: 16.05.1936 edat: 82 anys procedencia cex mateix hosp servei urologia data d'ingres 24.07.2018 data d'alta 25.07.2018 08:54:04 ates per ***************, *****; ****************, ****** informe d'alta d'hospitalitzacio motiu d'ingres paciente que ingresa de forma programada para realizacion de uretrotomia interna . antecedents alergia a penicilina y cloramfenicol . no habitos toxicos. antecedentes medicos: bloqueo auriculoventricular de primer grado hipertension arterial. diverticulosis extensa insuficiencia renal cronica colelitiasis antecedentes quirurgicos: exeresis de lesiones cutaneas con anestesia local protesis total de cadera cordectomia herniorrafia inguinal proces actual varon de 81a que a raiz de episodio de hematuria macroscopica se realiza cistoscopia que es

In [37]:
document1 = training_set[0]
negations = [result_element for result_element in document1["predictions"][0]["result"] if "NEG" in result_element["value"]["labels"]]
neg_scopes = [result_element for result_element in document1["predictions"][0]["result"] if "NSCO" in result_element["value"]["labels"]]
print(neg_scopes)

[{'value': {'start': 452, 'end': 468, 'labels': ['NSCO']}, 'id': 'ent1', 'from_name': 'label', 'to_name': 'text', 'type': 'labels'}, {'value': {'start': 844, 'end': 856, 'labels': ['NSCO']}, 'id': 'ent2', 'from_name': 'label', 'to_name': 'text', 'type': 'labels'}, {'value': {'start': 872, 'end': 895, 'labels': ['NSCO']}, 'id': 'ent4', 'from_name': 'label', 'to_name': 'text', 'type': 'labels'}, {'value': {'start': 1002, 'end': 1008, 'labels': ['NSCO']}, 'id': 'ent6', 'from_name': 'label', 'to_name': 'text', 'type': 'labels'}, {'value': {'start': 1543, 'end': 1576, 'labels': ['NSCO']}, 'id': 'ent8', 'from_name': 'label', 'to_name': 'text', 'type': 'labels'}, {'value': {'start': 1992, 'end': 2020, 'labels': ['NSCO']}, 'id': 'ent14', 'from_name': 'label', 'to_name': 'text', 'type': 'labels'}, {'value': {'start': 2157, 'end': 2188, 'labels': ['NSCO']}, 'id': 'ent16', 'from_name': 'label', 'to_name': 'text', 'type': 'labels'}, {'value': {'start': 2223, 'end': 2235, 'labels': ['NSCO']}, 'id':

In [44]:
tokens = [
    {
        "word": "historia",
        "word_lower": "historia",
        "is_capitalized": False,
        "is_all_caps": False,
        "is_digit": False,
        "word_length": 8,
        "contains_digits": False,
        "pos": "NOUN",
        "lemma": "historia",
        "start-end": {"start": 4, "end": 12},
        "prefix_2": "hi",
        "suffix_2": "ia",
        "previous_words": ["interna"],
        "next_word": "clinica",
    },
    {
        "word": "historia",
        "word_lower": "historia",
        "is_capitalized": False,
        "is_all_caps": False,
        "is_digit": False,
        "word_length": 8,
        "contains_digits": False,
        "pos": "NOUN",
        "lemma": "historia",
        "start-end": {"start": 449, "end": 452},
        "prefix_2": "hi",
        "suffix_2": "ia",
        "previous_words": ["interna"],
        "next_word": "clinica",
    },
    {
        "word": "historia",
        "word_lower": "historia",
        "is_capitalized": False,
        "is_all_caps": False,
        "is_digit": False,
        "word_length": 8,
        "contains_digits": False,
        "pos": "NOUN",
        "lemma": "historia",
        "start-end": {"start": 452, "end": 456},
        "prefix_2": "hi",
        "suffix_2": "ia",
        "previous_words": ["interna"],
        "next_word": "clinica",
    },
    {
        "word": "historia",
        "word_lower": "historia",
        "is_capitalized": False,
        "is_all_caps": False,
        "is_digit": False,
        "word_length": 8,
        "contains_digits": False,
        "pos": "NOUN",
        "lemma": "historia",
        "start-end": {"start": 467, "end": 478},
        "prefix_2": "hi",
        "suffix_2": "ia",
        "previous_words": ["interna"],
        "next_word": "clinica",
    },
    {
        "word": "historia",
        "word_lower": "historia",
        "is_capitalized": False,
        "is_all_caps": False,
        "is_digit": False,
        "word_length": 8,
        "contains_digits": False,
        "pos": "NOUN",
        "lemma": "historia",
        "start-end": {"start": 863, "end": 866},
        "prefix_2": "hi",
        "suffix_2": "ia",
        "previous_words": ["interna"],
        "next_word": "clinica",
    },
    {
        "word": "historia",
        "word_lower": "historia",
        "is_capitalized": False,
        "is_all_caps": False,
        "is_digit": False,
        "word_length": 8,
        "contains_digits": False,
        "pos": "NOUN",
        "lemma": "historia",
        "start-end": {"start": 866, "end": 872},
        "prefix_2": "hi",
        "suffix_2": "ia",
        "previous_words": ["interna"],
        "next_word": "clinica",
    },
    {
        "word": "historia",
        "word_lower": "historia",
        "is_capitalized": False,
        "is_all_caps": False,
        "is_digit": False,
        "word_length": 8,
        "contains_digits": False,
        "pos": "NOUN",
        "lemma": "historia",
        "start-end": {"start": 900, "end": 923},
        "prefix_2": "hi",
        "suffix_2": "ia",
        "previous_words": ["interna"],
        "next_word": "clinica",
    },
]

In [46]:

# Set BIO tags for the tokens, based on the training negations
def BIO_tagging(tokens, labels, label_name):
    """
    Set BIO tags for the tokens based on the labels from the training data.
    
    Args:
    - tokens (list): A list of token dictionaries
    - labels (list): A list of labels from the training data
    - label_name (str): The name of the label to use for tagging (e.g., "NEG")
    """
     
    negation_idx = 0

    for token in tokens:
        if 'tag' not in token:
            token['tag'] = 'O'

        if negation_idx >= len(labels):
            continue

        negation = labels[negation_idx]

        neg_start = negation['value']['start']
        neg_end = negation['value']['end']

        # Skip whitespace characters at the start and end positions
        if document1['data']['text'][neg_start] == ' ':
            neg_start += 1
        if document1['data']['text'][neg_end - 1] == ' ':
            neg_end -= 1

        token_start = token['start-end']['start']
        token_end = token['start-end']['end']

        if token_start == neg_start:
            token['tag'] = f'B-{label_name}'
        elif neg_start < token_start < neg_end:
            token['tag'] = f'I-{label_name}'
        elif token_start > neg_end:
            # Move to the next negation that starts after the current token
            while negation_idx < len(labels) and token_start > labels[negation_idx]['value']['start']:
                negation_idx += 1
                negation = labels[negation_idx]
            
            if token_start == negation['value']['start']:
                token['tag'] = f'B-{label_name}'

    # Print tokens with BIO tags
    for token in tokens:
        print(token)

In [47]:
BIO_tagging(tokens, negations, 'NEG')
print("--")
BIO_tagging(tokens, neg_scopes, 'NSCO')

{'word': 'historia', 'word_lower': 'historia', 'is_capitalized': False, 'is_all_caps': False, 'is_digit': False, 'word_length': 8, 'contains_digits': False, 'pos': 'NOUN', 'lemma': 'historia', 'start-end': {'start': 4, 'end': 12}, 'prefix_2': 'hi', 'suffix_2': 'ia', 'previous_words': ['interna'], 'next_word': 'clinica', 'tag': 'O'}
{'word': 'historia', 'word_lower': 'historia', 'is_capitalized': False, 'is_all_caps': False, 'is_digit': False, 'word_length': 8, 'contains_digits': False, 'pos': 'NOUN', 'lemma': 'historia', 'start-end': {'start': 449, 'end': 452}, 'prefix_2': 'hi', 'suffix_2': 'ia', 'previous_words': ['interna'], 'next_word': 'clinica', 'tag': 'B-NEG'}
{'word': 'historia', 'word_lower': 'historia', 'is_capitalized': False, 'is_all_caps': False, 'is_digit': False, 'word_length': 8, 'contains_digits': False, 'pos': 'NOUN', 'lemma': 'historia', 'start-end': {'start': 452, 'end': 456}, 'prefix_2': 'hi', 'suffix_2': 'ia', 'previous_words': ['interna'], 'next_word': 'clinica', 

In [31]:

# Set BIO tags for the tokens, based on the training negations

negation_idx = 0

for token in tokens:
    token['tag'] = 'O'

    if negation_idx >= len(negations):
        continue

    negation = negations[negation_idx]

    neg_start = negation['value']['start']
    neg_end = negation['value']['end']

    # Skip whitespace characters at the start and end positions
    if document1['data']['text'][neg_start] == ' ':
        neg_start += 1
    if document1['data']['text'][neg_end - 1] == ' ':
        neg_end -= 1

    token_start = token['start-end']['start']
    token_end = token['start-end']['end']

    if token_start == neg_start:
        token['tag'] = 'B-NEG'
    elif neg_start < token_start < neg_end:
        token['tag'] = 'I-NEG'
    elif token_start > neg_end:
        print("Token starts after negation")
        # Move to the next negation that starts after the current token
        while negation_idx < len(negations) and token_start > negations[negation_idx]['value']['start']:
            negation_idx += 1
            negation = negations[negation_idx]
        
        if token_start == negation['value']['start']:
            token['tag'] = 'B-NEG'

# Print tokens with BIO tags
for token in tokens:
    print(token)

Token starts after negation
Token starts after negation
{'word': 'historia', 'word_lower': 'historia', 'is_capitalized': False, 'is_all_caps': False, 'is_digit': False, 'word_length': 8, 'contains_digits': False, 'pos': 'NOUN', 'lemma': 'historia', 'start-end': {'start': 4, 'end': 12}, 'prefix_2': 'hi', 'suffix_2': 'ia', 'previous_words': ['interna'], 'next_word': 'clinica', 'tag': 'O'}
{'word': 'historia', 'word_lower': 'historia', 'is_capitalized': False, 'is_all_caps': False, 'is_digit': False, 'word_length': 8, 'contains_digits': False, 'pos': 'NOUN', 'lemma': 'historia', 'start-end': {'start': 449, 'end': 452}, 'prefix_2': 'hi', 'suffix_2': 'ia', 'previous_words': ['interna'], 'next_word': 'clinica', 'tag': 'B-NEG'}
{'word': 'historia', 'word_lower': 'historia', 'is_capitalized': False, 'is_all_caps': False, 'is_digit': False, 'word_length': 8, 'contains_digits': False, 'pos': 'NOUN', 'lemma': 'historia', 'start-end': {'start': 467, 'end': 478}, 'prefix_2': 'hi', 'suffix_2': 'ia',